In [42]:
import re

from collections import defaultdict
from itertools import combinations
from math import prod


def ints(text: str) -> list[int]:
    return [int(x) for x in re.findall("-?\\d+", text)]


def first(iterable):
    return next(iter(iterable), None)


def data(day: int, parser=str, sep="\n", example=False) -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    filename = f"2025/{day}-example.txt" if example else f"2025/{day}.txt"
    sections = open(filename).read().rstrip().split(sep)
    return [parser(section) for section in sections]

In [31]:
# Day 1: Secret Entrance
def rotate(rotations, dial=50):
    stops_at_zero, passes_zero = 0, 0
    for rotation in rotations:
        dir, steps = rotation[0], int(rotation[1:])
        passes_zero += steps // 100  # Full rotations
        if dir == "R":
            new_dial = (dial + steps) % 100
            if new_dial < dial and new_dial != 0:
                passes_zero += 1
        else:
            new_dial = (dial - steps) % 100
            if new_dial > dial and dial != 0:
                passes_zero += 1
        if new_dial == 0:
            stops_at_zero += 1
        dial = new_dial
    return stops_at_zero, passes_zero


stops, passes = rotate(data(1))
print(f"Part 1: {stops}")  # 995
print(f"Part 2: {stops + passes}")  # 5847

Part 1: 995
Part 2: 5847


In [32]:
# Day 2: Gift Shop
def invalid_ids(ranges, part1=True):
    def invalid(id):
        id = str(id)
        middle = len(id) // 2
        first_half = id[:middle]
        second_half = id[middle:]
        return first_half == second_half

    def invalid_part2(id):
        # Substrings from pos 1 -> middle, check if they are repeated for the
        # rest of the string
        id = str(id)
        for i in range(1, len(id) // 2 + 1):
            if len(id) % i == 0:
                repetitions = len(id) // i - 1
                if id[:i] * repetitions == id[i:]:
                    return True  # repeated pattern
        return False

    def start_stop(_range):
        start, stop = map(int, _range.split("-"))
        return range(start, stop + 1)

    if part1:
        return sum(id for _range in ranges for id in start_stop(_range) if invalid(id))
    else:
        return sum(
            id for _range in ranges for id in start_stop(_range) if invalid_part2(id)
        )


print(f"Part 1: {invalid_ids(data(2, sep=','))}")  # 56660955519
print(f"Part 2: {invalid_ids(data(2, sep=','), part1=False)}")  # 79183223243

Part 1: 56660955519
Part 2: 79183223243


In [33]:
# Day 3: Lobby
def max_jolt(bank, batteries_following, start_position=0):
    # find largest jolt value in bank from start_position with number of
    # batteries_following. Returns largest jolt and next start_position to start
    # from.
    jolt, position = 0, 0
    for pos, battery in enumerate(bank[start_position:], start_position):
        if int(battery) > jolt and pos + batteries_following < len(bank):
            jolt = int(battery)
            position = pos
    return jolt, position + 1


def max_battery(bank, number_of_batteries=2):
    result, start = 0, 0
    for batteries_following in range(number_of_batteries - 1, -1, -1):
        jolt, start = max_jolt(bank, batteries_following, start)
        result += 10**batteries_following * jolt
    return result


banks = data(3)
print(f"Part 1: {sum(max_battery(bank) for bank in banks)}")  # 17087
print(f"Part 2: {sum(max_battery(bank, 12) for bank in banks)}")  # 169019504359949


Part 1: 17087
Part 2: 169019504359949


In [34]:
# Day 4: Printing Department
def neighbors(point, grid):
    row, col = point
    potential = (
        (row - 1, col - 1),
        (row, col - 1),
        (row + 1, col - 1),
        (row - 1, col),
        (row + 1, col),
        (row - 1, col + 1),
        (row, col + 1),
        (row + 1, col + 1),
    )
    for r, c in potential:
        if 0 <= r < len(grid[0]) and 0 <= c < len(grid):
            yield (r, c)


def adjacent_paper_rolls(point, grid):
    return sum(1 for p in neighbors(point, grid) if grid[p[0]][p[1]] == "@")


def removable_rolls(grid):
    result = []
    for r, _ in enumerate(grid):
        for c, char in enumerate(grid[r]):
            p = (r, c)
            if char == "@" and adjacent_paper_rolls(p, grid) < 4:
                result += [p]
    return result


def remove_roll(point, grid):
    r, c = point
    row = list(grid[r])
    row[c] = "."
    grid[r] = "".join(row)
    return grid


def one_forklift_round(grid):
    removable = removable_rolls(grid)
    for p in removable:
        grid = remove_roll(p, grid)
    return len(removable), grid


grid = data(4)

print(f"Part 1: {len(removable_rolls(grid))}")  # 1346

rolls_removed_this_round, grid = one_forklift_round(grid)
total_rolls_removed = rolls_removed_this_round
while rolls_removed_this_round > 0:
    rolls_removed_this_round, grid = one_forklift_round(grid)
    total_rolls_removed += rolls_removed_this_round

print(f"Part 2: {total_rolls_removed}")  # 8493

Part 1: 1346
Part 2: 8493


In [35]:
# Day 5: Cafeteria
def is_fresh(id, ranges):
    def within_range(id, r):
        low, high = map(int, r.split("-"))
        return low <= id <= high

    return any(within_range(id, r) for r in ranges)


def merge(ranges_strings):
    ranges, merged = [], []
    for r in ranges_strings:
        start, end = map(int, r.split("-"))
        ranges.append((start, end))

    ranges.sort(key=lambda x: x[0])
    current_start, current_end = ranges[0]

    for next_start, next_end in ranges[1:]:
        # Check for overlap or adjacency:
        if next_start <= current_end + 1:
            # Overlap: extend range to the max of the two ends
            current_end = max(current_end, next_end)
        else:
            # No overlap/adjacency
            merged.append((current_start, current_end))
            current_start, current_end = next_start, next_end
    merged.append((current_start, current_end))
    return merged


def range_size(range):
    return range[1] - range[0] + 1


fresh_ranges, available_ids = data(5, sep="\n\n")
fresh_ranges = fresh_ranges.splitlines()
available_ids = ints(available_ids)
print(f"Part 1: {sum(1 for id in available_ids if is_fresh(id, fresh_ranges))}")  # 744
print(f"Part 2: {sum(range_size(r) for r in merge(fresh_ranges))}")  # 347468726696961


Part 1: 744
Part 2: 347468726696961


In [36]:
# Day 6: Trash Compactor
def column(col, grid):
    return [row[col] for row in grid]


def part1(lines):
    def value(col):
        values = map(int, col[:-1])
        if col[-1] == "*":
            return prod(values)
        return sum(values)

    inputs = [i.split() for i in lines]
    columns = len(inputs[0])
    return sum(value(column(c, inputs)) for c in range(columns))


def part2(lines):
    inputs = [list(row) for row in lines]

    columns = len(inputs[0])
    operators = inputs[-1]
    problem_positions = [pos for pos, char in enumerate(operators) if char in "+*"] + [
        columns + 1
    ]

    total = 0
    for current, next in zip(problem_positions, problem_positions[1:]):
        values = []
        for col in range(current, next - 1):
            values += [int("".join(column(col, inputs[:-1])))]
        if operators[current] == "+":
            total += sum(values)
        elif operators[current] == "*":
            total += prod(values)
    return total  # 9640641878593


print(f"Part 1: {part1(data(6))}")  # 6503327062445
print(f"Part 2: {part2(data(6))}")  # 9640641878593

Part 1: 6503327062445
Part 2: 9640641878593


In [37]:
# Day 7: Laboratories
def fire_beam(grid):
    timelines_at_position = defaultdict(int)
    timelines_at_position[grid[0].find("S")] = 1
    splits = 0
    for row in grid:
        new_timelines = timelines_at_position.copy()
        for tachyon, timelines in timelines_at_position.items():
            if row[tachyon] == "^":
                if timelines_at_position[tachyon]:
                    splits += 1
                new_timelines[tachyon - 1] += timelines
                new_timelines[tachyon + 1] += timelines
                new_timelines[tachyon] = 0
        timelines_at_position = new_timelines.copy()
    return splits, sum(timelines_at_position.values())


part1, part2 = fire_beam(data(7))
print(f"Part 1: {part1}")  # 1642
print(f"Part 2: {part2}")  # 47274292756692


Part 1: 1642
Part 2: 47274292756692


In [ ]:
def part1(boxes):
    unconnected = boxes.copy()
    circuits = []
    for pair in pairs(boxes)[:1000]:
        _, box1, box2 = pair
        if box1 in unconnected and box2 in unconnected:
            # print(f"New circuit: {box1} and {box2}")
            circuits.append({box1, box2})
            unconnected.remove(box1)
            unconnected.remove(box2)
        elif box1 not in unconnected and box2 not in unconnected:
            circuit1 = first(c for c in circuits if box1 in c)
            circuit2 = first(c for c in circuits if box2 in c)
            if circuit1 is not circuit2:
                # print(f"Merge circuits: {circuit1} and {circuit2}")
                circuit1.update(circuit2)
                circuits.remove(circuit2)
        else:
            # box1 or box2 needs to be added to an existing circuit
            if circuit := first(c for c in circuits if box1 in c):
                # print(f"Extend {circuit} with {box2}")
                circuit.add(box2)
                unconnected.remove(box2)
            elif circuit := first(c for c in circuits if box2 in c):
                # print(f"Extend {circuit} with {box1}")
                circuit.add(box1)
                unconnected.remove(box1)
    return prod(sorted([len(c) for c in circuits], reverse=True)[:3])


boxes = {tuple(x) for x in map(ints, data(8))}
print(f"Part 1: {part1(boxes)}")  # 57564

Part 1: 57564


In [ ]:
def connect(boxes, connections=1000):
    def distance(p1, p2):
        return (p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2 + (p1[2] - p2[2]) ** 2

    def pairs(boxes):
        result = []
        for box1, box2 in combinations(boxes, 2):
            result += [(distance(box1, box2), box1, box2)]
        # print(result)
        result.sort(key=lambda x: x)
        return result

    unconnected = boxes.copy()
    circuits = []

    for iterations, pair in enumerate(pairs(boxes)):
        _, box1, box2 = pair
        if box1 in unconnected and box2 in unconnected:
            # print(f"New circuit: {box1} and {box2}")
            circuits.append({box1, box2})
            unconnected.remove(box1)
            unconnected.remove(box2)
        elif box1 not in unconnected and box2 not in unconnected:
            circuit1 = first(c for c in circuits if box1 in c)
            circuit2 = first(c for c in circuits if box2 in c)
            if circuit1 is not circuit2:
                # print(f"Merge circuits: {circuit1} and {circuit2}")
                circuit1.update(circuit2)
                circuits.remove(circuit2)
        else:
            # box1 or box2 needs to be added to an existing circuit
            if circuit := first(c for c in circuits if box1 in c):
                # print(f"Extend {circuit} with {box2}")
                circuit.add(box2)
                unconnected.remove(box2)
            elif circuit := first(c for c in circuits if box2 in c):
                # print(f"Extend {circuit} with {box1}")
                circuit.add(box1)
                unconnected.remove(box1)
        # print(unconnected)
        if iterations == connections:
            part1 = prod(sorted([len(c) for c in circuits], reverse=True)[:3])

        if not unconnected:
            return part1, box1[0] * box2[0]


boxes = {tuple(x) for x in map(ints, data(8))}
part1, part2 = connect(boxes)
print(f"Part 1: {part1}")  # 57564
print(f"Part 2: {part2}")  # 133296744


Part 1: 57564
Part 2: 133296744
